In [2]:
import pandas as pd
import numpy as np
import operator
import re
from collections import Counter

search_term = 'ziek'
doc = pd.read_csv('sample_dataset2.csv', sep=';')

doc.head()

,zoekopdracht,datum,url,sentiment,type,discussielengte,views,auteur,volgers,invloed,GPS breedtegraad,GPS lengtegraad,bericht tekst,type bron,titel
0,griep src:twitter,2017-09-20 10:47,https://twitter.com/AtotenmetA/status/91042508...,NaN,post,NaN,146.0,AtotenmetA,146,0.7,NaN,NaN,Afvallen? Dan raad ik je echt deze monsterlijk...,twitter,NaN
1,griep src:twitter,2017-09-20 10:42,https://twitter.com/grieptweets/status/9104237...,NaN,post,NaN,131.0,grieptweets,131,0.5,NaN,NaN,#griep #nieuws Varkenmassagraven laatste voorb...,twitter,NaN
2,griep src:twitter,2017-09-20 10:24,https://twitter.com/busy3978/status/9104192735...,NaN,post,NaN,648.0,busy3978,648,0.6,NaN,NaN,Me griep is nog steeds aanwezig nu al bijna we...,twitter,NaN
3,griep src:twitter,2017-09-20 10:09,https://twitter.com/_Sittard/status/9104156667...,NaN,post,2.0,420.0,_Sittard,420,1.0,NaN,NaN,"#RIVM: griepprik voorkomt griep, maar niet het...",twitter,NaN
4,griep src:twitter,2017-09-20 10:05,https://twitter.com/SGNieuws/status/9104145307...,NaN,post,NaN,2509.0,SGNieuws,2509,1.5,NaN,NaN,"Griepprik voorkomt griep, maar niet het aantal...",twitter,NaN


# Cleaning data
Here we do several things:
* remove retweets
* convert text to lowercase

In the context of our research a retweet does not add any relevant data. It actually skews the results since a specific phrasing get's repeated more often than it naturally would.

In [3]:
doc['bericht tekst'] = doc['bericht tekst'].fillna('')

In [4]:
doc['bericht tekst'] = doc['bericht tekst'].str.lower()

In [5]:
doc  = doc[~doc['bericht tekst'].str.contains('rt')]
# # doc[~doc['bericht tekst'].str.contains('rt')]
# doc.head()

In [6]:
doc = doc.drop_duplicates()

# Defenition of helper function


In [6]:
re_clean = re.compile(r'(https?://\S+|@\S+)')
re_words = re.compile(r'(\w+-?\w*)')

def clean_text(text: str):
    words = []
    if text:
        text = re_clean.sub(' ', text)
        words = re_words.findall(text)
    return words

In [7]:
all_tweets = pd.read_csv('sample_alltweets.csv', sep=';')
all_tweets.head()

FileNotFoundError: File b'sample_alltweets.csv' does not exist

# Cleaning data
Here we do several things:
* convert messages with no content with to the message `''`
* remove retweets
* convert to lowercase

The conversion is needed since the `clean_text()` function expects a string.

In [8]:
all_tweets['bericht tekst'] = all_tweets['bericht tekst'].fillna('')

In [9]:
all_tweets['bericht tekst'] = all_tweets['bericht tekst'].str.lower()

In [10]:
all_tweets = all_tweets[~all_tweets['bericht tekst'].str.contains('rt')]
all_tweets.head()

,zoekopdracht,datum,url,sentiment,type,discussielengte,views,auteur,volgers,invloed,GPS breedtegraad,GPS lengtegraad,bericht tekst,type bron,titel
3,src:twitter,2017-06-14 23:59,https://twitter.com/Neraajz/status/87511059080...,NaN,post,NaN,1548.0,Neraajz,1548,2.9,NaN,NaN,overmorgen examen en volgende week ook en dan ...,twitter,NaN
4,src:twitter,2017-06-14 23:59,https://twitter.com/pepsi_elwin/status/8751105...,NaN,comment,NaN,NaN,pepsi_elwin,250,1.0,NaN,NaN,@redwiff that probly requires hacking your swi...,twitter,NaN
6,src:twitter,2017-06-14 23:59,https://twitter.com/KvW_beach/status/875110593...,NaN,post,NaN,71.0,KvW_beach,71,0.6,NaN,NaN,morgen begint het tornooi voor ons in den haag...,twitter,NaN
7,src:twitter,2017-06-14 23:59,https://twitter.com/rene_fiala/status/87511059...,+,comment,NaN,NaN,rene_fiala,29,0.3,NaN,NaN,@kivicod @stoneemlg wtf hahaha 😂,twitter,NaN
9,src:twitter,2017-06-14 23:59,https://twitter.com/5HOfficialMex/status/87511...,NaN,post,3.0,27834.0,5HOfficialMex,27834,1.7,NaN,NaN,normani en el snapchat de @joshleviworld #3 ht...,twitter,NaN


In [14]:
all_tweets['bericht woorden'] = all_tweets['bericht tekst'].map(clean_text)

counter = Counter()

for words in all_tweets['bericht woorden']:
    counter.update(words)
counter.most_common(300)

[('de', 2968),
 ('in', 2469),
 ('is', 2443),
 ('ik', 2419),
 ('een', 2219),
 ('en', 2172),
 ('van', 1699),
 ('het', 1695),
 ('the', 1639),
 ('je', 1542),
 ('op', 1330),
 ('a', 1324),
 ('i', 1324),
 ('dat', 1256),
 ('to', 1182),
 ('met', 1103),
 ('voor', 1086),
 ('niet', 1080),
 ('of', 1015),
 ('s', 968),
 ('you', 899),
 ('te', 819),
 ('maar', 789),
 ('and', 780),
 ('die', 775),
 ('me', 714),
 ('zijn', 675),
 ('it', 673),
 ('ook', 657),
 ('for', 623),
 ('er', 593),
 ('on', 587),
 ('dan', 551),
 ('aan', 542),
 ('wat', 538),
 ('als', 523),
 ('via', 522),
 ('we', 517),
 ('t', 516),
 ('nog', 489),
 ('was', 485),
 ('wel', 481),
 ('zo', 476),
 ('bij', 472),
 ('om', 468),
 ('heb', 443),
 ('mijn', 438),
 ('ben', 437),
 ('over', 435),
 ('2', 428),
 ('nu', 422),
 ('ze', 422),
 ('my', 421),
 ('this', 419),
 ('dit', 419),
 ('that', 412),
 ('1', 367),
 ('kan', 359),
 ('al', 344),
 ('naar', 341),
 ('so', 339),
 ('geen', 327),
 ('haar', 323),
 ('with', 320),
 ('ja', 315),
 ('weer', 310),
 ('m', 310),


In [12]:
common_words = [word[0] for word in counter.most_common(300)]
common_words = set(common_words)
common_words

['de',
 'in',
 'is',
 'ik',
 'een',
 'en',
 'van',
 'het',
 'the',
 'je',
 'op',
 'a',
 'i',
 'dat',
 'to',
 'met',
 'voor',
 'niet',
 'of',
 's',
 'you',
 'te',
 'maar',
 'and',
 'die',
 'me',
 'zijn',
 'it',
 'ook',
 'for',
 'er',
 'on',
 'dan',
 'aan',
 'wat',
 'als',
 'via',
 'we',
 't',
 'nog',
 'was',
 'wel',
 'zo',
 'bij',
 'om',
 'heb',
 'mijn',
 'ben',
 'over',
 '2',
 'nu',
 'ze',
 'my',
 'this',
 'dit',
 'that',
 '1',
 'kan',
 'al',
 'naar',
 'so',
 'geen',
 'haar',
 'with',
 'ja',
 'weer',
 'm',
 '3',
 'echt',
 'uit',
 'be',
 'by',
 'u',
 'no',
 'at',
 'video',
 'your',
 'heeft',
 'not',
 'door',
 'meer',
 'man',
 '0',
 'gaat',
 'leuk',
 'just',
 'n',
 'are',
 'vind',
 'what',
 'toch',
 'omg',
 'deze',
 'do',
 'wil',
 'can',
 'nieuws',
 'moet',
 'goed',
 'have',
 'mij',
 'new',
 'but',
 'one',
 'hoe',
 'like',
 'all',
 'veel',
 'hij',
 'he',
 'love',
 'na',
 'jaar',
 'now',
 'even',
 '5',
 'nl',
 'hebben',
 'want',
 'mensen',
 'as',
 '4',
 'out',
 'dus',
 'vrouw',
 'zou',
 '

In [13]:
blacklisted_words = common_words
blacklisted_words.add(search_term)

AttributeError: 'list' object has no attribute 'add'

In [ ]:
doc['bericht woorden'] = doc['bericht tekst'].map(clean_text)
counter = Counter()

for words in doc['bericht woorden']:
    words = set(words)
    filtered_words = words - blacklisted_words
    counter.update(filtered_words)
    
related_words = counter.most_common(100)
related_words

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
most_related_word = related_words[0][0]
print('Word: {}'.format(most_related_word))
doc[doc['bericht tekst'].str.contains(most_related_word)].filter(items=['bericht tekst', 'auteur', 'type'])

In [7]:
word = related_words[4][0]
print('Word: {}'.format(word))
doc[doc['bericht tekst'].str.contains(word)].filter(items=['bericht tekst', 'auteur', 'type'])

NameError: name 'related_words' is not defined

In [34]:
import difflib
from functools import reduce


def get_similarity(values: pd.Series):
    if len(values) <= 1:
        return 0
    values = values.to_frame()
    values['id'] = values.index
    similarity_total = []
    for _, row1 in values.iterrows():
        for _, row2 in values.iterrows():
            if row1['id'] == row2['id']:
                continue
            sim = difflib.SequenceMatcher(None, row1[0], row2[0])
            similarity_total.append(sim.real_quick_ratio())
    
    if not similarity_total:
        return 0
    return reduce(lambda x, y: x + y, similarity_total) / len(similarity_total)


sim = doc.groupby(['auteur'])['bericht tekst'].agg(get_similarity)
sim[:30]

auteur
1202Nathasja       0.000000
16Klepper          0.000000
45_tyger           0.000000
ACHelfensteijn     0.000000
AadvanTongeren     0.000000
Abel_Hakeem        0.000000
Alentejowijnen     0.000000
Algra              0.939535
AlphatronPR        0.000000
AltNieuws          0.000000
AnettePartini      0.000000
AnneMiekVroom      0.000000
Annemieke2801      0.000000
Annemiekhop        0.000000
Annette17056451    0.000000
AntoinetNouveau    0.000000
ArtyAnniD          0.992754
AshleesProdz       0.000000
AtotenmetA         0.000000
Avonddame          0.000000
BBooster           0.000000
BYFITNL            0.000000
BadMoFoNL          1.000000
Beau_BOA           0.000000
BeautyxSpace       0.000000
Benniestaut        0.839005
Bepvankleij        0.000000
BergsmaGert        0.000000
BerylDerosia       0.000000
BidwIdn            0.000000
Name: bericht tekst, dtype: float64

In [35]:
sim[sim > 0.95]  # list of authors to blacklist

auteur
ArtyAnniD          0.992754
BadMoFoNL          1.000000
Elk_Uur_Een_Mop    1.000000
Endimi             0.993548
EversJette         0.974743
Give_in2me         0.982143
Jezzebellz         0.957655
JoopEding          0.989011
JuffrouwPrut       0.990291
MRDenDekker        0.966321
MarinaMakeenko     0.956522
Rina_v_R           0.970643
TommyfunnyAA       0.982659
TwitziZilan        0.987342
Vlooienbaalcom     0.972644
bedrijfsnet        1.000000
busy3978           0.988848
debrackeleer       0.984848
grieptweets        0.969381
irenemuller_SA     0.952381
jtmooten           0.988764
nomnoom            1.000000
vragenrubriek      1.000000
Name: bericht tekst, dtype: float64